```{autolink-concat}
```

::::{margin}
:::{card} Investigation of acceptance matrix with Dalitz-plot decomposition
TR-035
:::
::::

# DPD acceptance matrix

In [ ]:
from __future__ import annotations

import numpy as np
from IPython.display import HTML, display
from jpsi_nstar.io import mute_jax_warnings, mute_tensorflow_warnings

np.set_printoptions(linewidth=120)
mute_jax_warnings()
mute_tensorflow_warnings()

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))


To speed up the fit the part of the negative log Liklihood (NLL) function representing the normalization computed over the  Monte-Carlo phsp can be pre-computed. Note that this method can only be applied in the _specific case that masses and widths are fixed_ as the normalization depends on these parameters.

Statistically this approach can be derived as follows:
Given total sample $\vec{x}$ consisting of _N_ independent observations of a set the $n$ quantities $x$ depending the Liklihood function can be written as the product of the (Probability Density Functions) PDFs $f$ of each single observation:

$$L(\vec{x} ; \vec{\theta})=\prod_{\mathfrak{i}=1}^N f\left(x_i ; \vec{\theta}\right)$$

$\vec{\theta}$ represents the set of $m$ unknown parameters the chosen PDF depends on.
In the case of the this amplitude analysis the PDFs are the normalized intensity functions for each partial wave:

$$f(x_i ; \vec{\theta})= \frac{I(x_i ; \vec{\theta})}{\int I(\vec{x} ; \vec{\theta}) d\vec{x}}$$ 

 The $x_i$ represent the selected events containing the information about the Mandelstam variables and the decay and production angles. With the expression $f$ the NLL function is given as:

$$
NLL(x_i ; \vec{\theta})=-\sum^n_{i=1} \log \left(\frac{I(x_i ; \vec{\theta})}{\int I(\vec{x} ; \vec{\theta}) d\vec{x}}\right) = \underbrace{-\sum^n_{i=1} \log(I(x_i ; \vec{\theta}))}_{\mathrm{Data}} + n\cdot \log\left(\int I(\vec{x} ; \vec{\theta}) d\vec{x} \right)
$$

Since the second term integrates over all phase-space points, it is independent of \( x_i \) and can be approximated using Monte Carlo integration:

$$
\int I(\vec{x} \,; \vec{\theta}) \, d\vec{x} = 
c^{\dagger} \cdot \frac{1}{N_{\mathrm{MC}}} \sum_{j=1}^{N_{\mathrm{MC}}} I(\vec{x}_j^{\mathrm{MC}} \,; \vec{\theta}) \cdot c 
\cdot \left( \int d\vec{x} \right)
$$

The term $\frac{1}{N_{\mathrm{MC}}} \sum_{j=1}^{N_{\mathrm{MC}}} I(\vec{x}_j^{\mathrm{MC}} ; \vec{\theta})$ represents the so-called acceptance matrix, a hermitian matrix which we call $X$ from now on. The acceptance matrix can be used obtain integrated intensity via the **bilinear relation**:

$$
\int I\\ d\vec{x}=c^{\dagger}\cdot X \cdot c
$$
## Construction of the acceptance matrix
The acceptance matrix can be constructed by probing with four different values for $c_ij$  leading to the four equations:
$$
A_{ij}^{+}=\int I(c_i=1, c_j=1) \\ d\vec{x} =  \left[ {\begin{array}{cc}
   1 & 1 \\
  \end{array} } \right]\cdot   \left[ {\begin{array}{cc}
   X_{ii} & X_{ij} \\
   X_{ji}& X_{jj} \\
  \end{array} } \right] \cdot  \left[ {\begin{array}{cc}
   1 \\
1
  \end{array} } \right]
$$

$$
A_{ij}^{-}=\int I(c_i=1, c_j=-1) \\ d\vec{x} =  \left[ {\begin{array}{cc}
   1 & -1 \\
  \end{array} } \right]\cdot   \left[ {\begin{array}{cc}
   X_{ii} & X_{ij} \\
   X_{ji}& X_{jj} \\
  \end{array} } \right] \cdot  \left[ {\begin{array}{cc}
   1 \\
-1
  \end{array} } \right]
$$

$$
B_{ij}^{+}=\int I(c_i=1, c_j=i)\\ d\vec{x} =  \left[ {\begin{array}{cc}
   1 & i \\
  \end{array} } \right]\cdot   \left[ {\begin{array}{cc}
   X_{ii} & X_{ij} \\
   X_{ji}& X_{jj} \\
  \end{array} } \right] \cdot  \left[ {\begin{array}{cc}
   1 \\
i
  \end{array} } \right]
$$

$$
B_{ij}^{-}=\int I(c_i=1, c_j=-i) \\ d\vec{x} =  \left[ {\begin{array}{cc}
   1 & -i \\
  \end{array} } \right]\cdot   \left[ {\begin{array}{cc}
   X_{ii} & X_{ij} \\
   X_{ji}& X_{jj} \\
  \end{array} } \right] \cdot  \left[ {\begin{array}{cc}
   1 \\
-i
  \end{array} } \right]
$$
The $A_{ij}$ and $B_{ij}$ are the results of the integration over the phase space for the $c_i$ and $c_j$. The equations can be solved for the elements of $X$ by rewriting them in matrix form:
The elements of $X$ can therefor be expressed as:

$$
X_{ij} = \frac{(A_{ij}^{+}-A_{ij}^{-})}{4}+i\cdot\frac{(B_{ij}^{+}-B_{ij}^{-})}{4}
$$
## Speed up the construction of $X$
Using four equations as mentioned above is not computational efficient as intensity matrix for all four combinations of $c_i$ and $c_j$ has to be computed. However the hermitian property of the acceptance matrix can be used to reduce the number of equations necessary to construct the matrix. A hermitian matrix is mirror symmetric with respect to its main diagonal, up to the complex conjugation of all entries. Therefore one can only calculate the upper triangle of $X$ and then fill the lower triangle with the complex conjugate. As probes the vectors The diagonal of $X$ is real valued and can be computed using the bilinear relation with $c_i=c_j=1$.